In [97]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["nutriwiser_db"]
collection = db["additifs_scraped"]

# Obtenir toutes les valeurs distinctes d'un champ
distinct_names = collection.distinct("danger")
distinct_classes = collection.distinct("additive_classes")

print(distinct_names)
print(distinct_classes)

['', 'Douteux', 'Ne pas abuser', 'Pas/peu toxique', 'Toxique', 'Très toxique']
['antioxidant', 'carrier', 'coagulant', 'colour', 'emulsifier', 'flavour-enhancer', 'humectant', 'preservative', 'propellent-gas', 'sequestrant', 'stabiliser', 'sweetener', 'thickener']


In [98]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="nutriwiser",
    password="nutriwiser",
    database="nutriwiser_db"
)

cursor = conn.cursor()

query = "INSERT INTO classes (label) VALUES (%s)"

for classe in distinct_classes:
    print(classe)
    classe = classe.lower().strip()
    cursor.execute(query, (classe,))

conn.commit()
cursor.close()
conn.close()


antioxidant
carrier
coagulant
colour
emulsifier
flavour-enhancer
humectant
preservative
propellent-gas
sequestrant
stabiliser
sweetener
thickener


In [99]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="nutriwiser",
    password="nutriwiser",
    database="nutriwiser_db"
)

cursor = conn.cursor()

query = "INSERT INTO toxicite (label) VALUES (%s)"

for names in distinct_names:
    names = names.lower().strip()
    print(names)
    cursor.execute(query, (names,))

conn.commit()
cursor.close()
conn.close()


douteux
ne pas abuser
pas/peu toxique
toxique
très toxique


In [100]:
import mysql.connector
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")

result = client['nutriwiser_db']['additifs_scraped'].find({})


conn = mysql.connector.connect(
    host="localhost",
    user="nutriwiser",
    password="nutriwiser",
    database="nutriwiser_db"
)

cursor = conn.cursor()

for doc in result:
    try:
        code = doc.get('additive_code')
        if not code:
            continue

        names = doc.get('names') or []
        label_toxicite = (doc.get('danger') or '').strip()
        additive_classes = doc.get('additive_classes', [])
        description = doc.get('description')
        description_avancee = doc.get('description_avancee')
        remarques = doc.get('remarques')

        # Vérifier ou insérer la toxicité
        cursor.execute("SELECT id FROM toxicite WHERE label = %s", (label_toxicite,))
        row = cursor.fetchone()
        if not row:
            cursor.execute("INSERT INTO toxicite (label) VALUES (%s)", (label_toxicite,))
            conn.commit()
            toxicite_id = cursor.lastrowid
        else:
            toxicite_id = row[0]

        # Insérer dans additifs
        cursor.execute("""
            INSERT INTO additifs (code, toxicite_id, description, description_avancee, remarques)
            VALUES (%s, %s, %s, %s, %s)
        """, (code, toxicite_id, description, description_avancee, remarques))

        # Insérer dans additifs_noms
        for name in names:
            cursor.execute("""
                INSERT INTO additifs_noms (code_additif, nom)
                VALUES (%s, %s)
            """, (code, name))

        # Insérer dans classes & additifs_classes
        for class_label in additive_classes:
            # Vérifier ou insérer la classe
            cursor.execute("SELECT id FROM classes WHERE label = %s", (class_label,))
            row = cursor.fetchone()
            if not row:
                cursor.execute("INSERT INTO classes (label) VALUES (%s)", (class_label,))
                conn.commit()
                class_id = cursor.lastrowid
            else:
                class_id = row[0]

            # Créer le lien additif <-> classe
            cursor.execute("""
                INSERT INTO additifs_classes (code_additif, class_id)
                VALUES (%s, %s)
            """, (code, class_id))

        conn.commit()
        print(f"[OK] Additif inséré : {code}")

    except mysql.connector.Error as err:
        print(f"[ERREUR MySQL] sur {code} :", err)
        conn.rollback()


[OK] Additif inséré : E100
[OK] Additif inséré : E101
[OK] Additif inséré : E102
[OK] Additif inséré : E104
[OK] Additif inséré : E105
[OK] Additif inséré : E107
[OK] Additif inséré : E110
[OK] Additif inséré : E111
[OK] Additif inséré : E120
[OK] Additif inséré : E121
[OK] Additif inséré : E122
[OK] Additif inséré : E123
[OK] Additif inséré : E124
[OK] Additif inséré : E125
[OK] Additif inséré : E126
[OK] Additif inséré : E127
[OK] Additif inséré : E128
[OK] Additif inséré : E129
[OK] Additif inséré : E130
[OK] Additif inséré : E131
[OK] Additif inséré : E132
[OK] Additif inséré : E133
[OK] Additif inséré : E140
[OK] Additif inséré : E141
[OK] Additif inséré : E142
[OK] Additif inséré : E143
[OK] Additif inséré : E150a
[OK] Additif inséré : E150b
[OK] Additif inséré : E150c
[OK] Additif inséré : E150d
[OK] Additif inséré : E151
[OK] Additif inséré : E152
[OK] Additif inséré : E153
[OK] Additif inséré : E154
[OK] Additif inséré : E155
[OK] Additif inséré : E160a
[OK] Additif inséré : E

In [32]:

filter={}

result = client['nutriwiser_db']['additifs_scraped'].find(
  filter=filter,
  limit=1
)


for doc in result:
    
    Code = doc.get('code')
    
    niveau = doc.get('danger')
    
    # if Code:
    #     cursor.execute("UPDATE additifs SET toxicite = %s WHERE code = %s", (niveau, Code))


In [89]:
result2 = client['nutriwiser_db']['additifs'].find(
  filter=filter,
)